# M2 TIDE : Application des connaissances

In [ ]:
"""TODO (dimanche)
- déposer projet git complet sur un repo
- créer un repo sans la correction (j'irai push la correction en live pendant le cours)
"""

"TODO (dimanche)\n- Simplifier/raccourcir questions + ajouter questions sur les parties 6 et 7\n- créer un fichier tools.py\n- vérifier commentaires fonctions créées\n- déposer projet git complet sur un repo\n- créer un repo sans la correction (j'irai push la correction en live pendant le cours)\n"

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import StandardScaler
sys.path.insert(0, str(Path(os.getcwd()).resolve().parent.parent))
path = Path(f"../../data/processed").resolve()
from src.utils.class_modeling import ModelComparator

list_var_num: list = ['Duree_credit', 'Montant_credit', 'Age']
list_var_cat: list = ['Objet_credit', 'Historique_credit', 'Epargne', 'Anciennete_emploi', 
           'Situation_familiale', 'Anciennete_domicile', 'Nb_credits', 'Biens', 
           'Type_emploi', 'Comptes', 'Taux_effort', 'Autres_credits', 'Statut_domicile', 
           'Garanties', 'Telephone', 'Nb_pers_charge', 'Etranger']
target: str = "Cible"

X_train = pd.read_parquet(f"{path}/X_train.parquet")
X_test = pd.read_parquet(f"{path}/X_test.parquet")
y_train = pd.read_parquet(f"{path}/y_train.parquet").squeeze()
y_test = pd.read_parquet(f"{path}/y_test.parquet").squeeze()
random_state = 0

### 6. Régressions logistiques


<div class="alert alert-block alert-info">
<b>🔔 Rappel sur la régression logistique :</b></p>
  <table>
    <thead>
      <tr>
        <th>Avantages</th>
        <th>Inconvénients</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>
          <ul>
            <li><code>interprétabilité des résultats</code> (signe des coefficients, rapports de cotes, etc…). Ces avantages rendent la RL particulièrement intéressante dans le monde professionnel. Par exemple, pour un score d'octroi de crédit, on a pour obligation légale d'expliquer les facteurs qui ont entrainé le refus.</li>
            <li><code>robustesse des résultats</code> (évite le sur-apprentissage) contrairement à un modèle Boosting (ex : XGBoost) ou Réseau de neurones.</li>
            <li><code>temps de calculs</code> (à l'apprentissage et à l'utilisation en production) plus faible qu'un XGBoost par exemple.</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Elle nécessite une <code>phase de préparation des données plus exigeante</code> que les modèles à base d'arbres et les réseaux de neurones (présélection des variables pour éviter les colinéarités, normalisation des variables explicatives continues pour éviter les effets d'échelle). Néanmoins, pour réduire la multicolinéarité et l'effet d'échelle, il est possible de pénaliser (L2:Ridge, L1:Lasso, L1+L2:Elastic net).</li>
            <li><code>moins performante</code></li>
          </ul>
        </td>
      </tr>
    </tbody>
  </table>
<div>

#### 6.1 Identifier les variables explicatives non pertinentes pour la régression logistique (cf parties précédentes sur l'étude des liens)

In [2]:
list_var_cat_to_drop: list = ['Statut_domicile', 'Taux_effort', 'Nb_credits', 'Type_emploi', 'Telephone', 'Anciennete_domicile', 'Nb_pers_charge'] # cf "Data_exploration.ipynb"
print(f'Rappel des variables à supprimer pour la régression logistique : {list_var_cat_to_drop}')
# => pas de variable catégorielle à supprimer

Rappel des variables à supprimer pour la régression logistique : ['Statut_domicile', 'Taux_effort', 'Nb_credits', 'Type_emploi', 'Telephone', 'Anciennete_domicile', 'Nb_pers_charge']


<div class="alert alert-block alert-info">
<b>Note :</b> On a supprimé les variables catégorielles d'origine de notre dataframe. Il va falloir trouver un moyen d'identifier les 
dummy variables associées
</div>

In [3]:
dummy_var = [i for i in X_train.columns if i.startswith('DUMMY_')]
dummy_to_drop = []
for i in list_var_cat_to_drop:
    for j in dummy_var:
        if i in j:dummy_to_drop.append(j)
dummy_to_drop_for_RL = list(set(dummy_to_drop))
print(dummy_to_drop_for_RL)

['DUMMY_Telephone_Oui__enregistré_sous_le_nom_du_client', 'DUMMY_Statut_domicile_Locataire', 'DUMMY_Statut_domicile_Hébergement_gratuit', 'DUMMY_Nb_credits_2', 'DUMMY_Nb_credits_1', 'DUMMY_Nb_pers_charge_2', 'DUMMY_Taux_effort_3', 'DUMMY_Taux_effort_1', 'DUMMY_Anciennete_domicile_1', 'DUMMY_Anciennete_domicile_4', 'DUMMY_Type_emploi_Chômeur___Non_qualifié___Non_résident', 'DUMMY_Type_emploi_Employé_qualifié___Fonctionnaire', 'DUMMY_Taux_effort_4', 'DUMMY_Type_emploi_Cadre___Indépendant___Employé_hautement_qualifié___Dirigeant', 'DUMMY_Nb_pers_charge_1', 'DUMMY_Anciennete_domicile_2', 'DUMMY_Telephone_Néant', 'DUMMY_Statut_domicile_Propriétaire', 'DUMMY_Anciennete_domicile_3', 'DUMMY_Nb_credits_3', 'DUMMY_Type_emploi_Non_qualifié___résident', 'DUMMY_Nb_credits_4', 'DUMMY_Taux_effort_2']


#### 6.2 Normaliser les variables explicatives continues

<div class="alert alert-block alert-info">
<b>🔔 Rappel : intérêts de la normalisation des variables explicatives continues</b></p>
  <table>
    <thead>
      <tr>
        <th>Avantage</th>
        <th>Description</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Échelle homogène</td>
        <td>Mettre toutes les variables sur une échelle comparable pour éviter qu'une variable ne domine l'apprentissage en raison de valeurs plus grandes.</td>
      </tr>
      <tr>
        <td>Amélioration de la convergence</td>
        <td>Accélère la convergence et facilite l'optimisation des algorithmes comme la descente de gradient, la régression linéaire ou les réseaux de neurones.</td>
      </tr>
      <tr>
        <td>Performance des algorithmes</td>
        <td>Évite que des variables à grande échelle biaisent les calculs, notamment pour les algorithmes sensibles aux distances (kNN, SVM, etc.).</td>
      </tr>
      <tr>
        <td>Interprétabilité</td>
        <td>Facilite l'interprétation de l'importance relative de chaque variable dans le modèle.</td>
      </tr>
    </tbody>
  </table>
</div>


<div class="alert alert-block alert-info">
  <b>⚠️ Remarque très importante :</b>
  il est primordial de normaliser en prenant pour base la table d'apprentissage. Puis, appliquer la normalisation sur
le jeu de données test
</div>


In [4]:
scaler = StandardScaler()

X_train_with_norm = X_train.copy()
X_test_with_norm = X_test.copy()

# Apprentissage pour normaliser sur le jeu d'apprentissage
scaler.fit(X_train[list_var_num]) # calcule la moyenne et l'écart-type des variables de la liste "list_var_num"

# Application de la normalisation sur les jeux de données (train et test)
X_train_with_norm[list_var_num]= scaler.transform(X_train[list_var_num]) # train
X_test_with_norm[list_var_num] = scaler.transform(X_test[list_var_num]) # test

#### 6.3 Recherche du meilleur modèle 

In [5]:
list_models = ["Logistic Regression (L1)", "Logistic Regression (L2)", "Logistic Regression (ElasticNet)"] # liste des modélisations à tester
logistic_models = ModelComparator(y_train= y_train, X_train= X_train_with_norm, y_test= y_test, X_test= X_test_with_norm, list_models= list_models, random_state= random_state) # initier la class permettant la modélisation

<div class="alert alert-block alert-info">
<b>🔔 Rappel : plusieurs approches pour optimiser les hyperparamètres d'un modèle :</b></p>
  <table>
    <thead>
      <tr>
        <th>Méthode</th>
        <th>Description</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Grid Search</td>
        <td>Exploration exhaustive d'une grille pré-définie de valeurs d'hyperparamètres pour trouver la meilleure combinaison.</td>
      </tr>
      <tr>
      <tr>
        <td>Méthode aléatoire</td>
        <td>Exploration de l'espace des hyperparamètres en échantillonnant aléatoirement des combinaisons, souvent plus efficace que Grid Search dans de grands espaces.</td>
      </tr>
        <td>Optimisation Bayésienne</td>
        <td> modèle probabiliste (souvent un processus gaussien) qui sélectionne intelligemment les prochaines combinaisons d'hyperparamètres à tester. Cela permet d'équilibrer exploration (essayer de nouvelles zones) et exploitation (affiner les zones prometteuses), réduisant ainsi le nombre d'évaluations nécessaires pour trouver des hyperparamètres optimaux.</td>
      </tr>
    </tbody>
  </table>
</div>


Nous allons utiliser `l'Optimisation Bayésienne` car c'est souvent celle qui permet les meilleurs résultats (mais elle peut être coûteuse en temps de calculs)

<div class="alert alert-block alert-info">
<b>🔔 Rappel : métriques candidates pour optimiser les performances du modèle :</b></p>
  <table>
    <thead>
      <tr>
        <th>Métrique</th>
        <th>Avantages</th>
        <th>Inconvénients</th>
        <th>Contexte d'utilisation</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Accuracy</td>
        <td>
          <ul>
            <li>Simple à interpréter</li>
            <li>Fournit une vue d'ensemble de la performance du modèle</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Peut être trompeuse en cas de classes très déséquilibrées (cas fréquent en scoring de crédit)</li>
            <li>Ne distingue pas le coût entre octroi de crédit à risque et refus injustifié</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Peut être utilisée comme indicateur global lorsque les défauts sont suffisamment représentatifs</li>
          </ul>
        </td>
      </tr>
      <tr>
        <td>Precision</td>
        <td>
          <ul>
            <li>Réduit le risque d'octroyer un crédit à un client à risque</li>
            <li>Aide à limiter les faux positifs, c'est-à-dire les cas où un client est classé comme bon payeur à tort</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Peut négliger les faux négatifs (clients bons payeurs à qui on refuse le crédit à tort)</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Privilégiée lorsque le coût d'un crédit accordé à un mauvais payeur est très élevé</li>
          </ul>
        </td>
      </tr>
      <tr>
        <td>Recall (Sensibilité)</td>
        <td>
          <ul>
            <li>Maximise la détection des clients à risque</li>
            <li>Diminue le nombre de défauts non identifiés</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Peut augmenter le nombre de faux positifs, entraînant le refus de bons clients</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Essentielle quand il est primordial d'identifier tous les cas à risque, même au prix d'un taux de refus plus élevé</li>
          </ul>
        </td>
      </tr>
      <tr>
        <td>F1-score</td>
        <td>
          <ul>
            <li>Fournit un équilibre entre la précision et le recall</li>
            <li>Utile pour mesurer la performance globale quand il faut minimiser à la fois les erreurs de crédit risqué accordé et les refus injustifiés</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Moins intuitif à interpréter pour les équipes non techniques</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Recommandé quand il faut trouver un compromis entre les faux positifs et faux négatifs</li>
          </ul>
        </td>
      </tr>
      <tr>
        <td>ROC AUC</td>
        <td>
          <ul>
            <li>Mesure la capacité du modèle à distinguer les bons payeurs des mauvais payeurs sur l'ensemble des seuils</li>
            <li>Indépendante du seuil de décision, ce qui est utile pour comparer plusieurs modèles</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Quand cible très déséquibrée, surestimation de la performance : un modèle peut obtenir une AUC élevée en se focalisant principalement sur la classe majoritaire, même s'il détecte mal la classe minoritaire. Ainsi, le modèle risque de mal détecter les mauvais payeurs</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Idéal pour la sélection et l'évaluation globale des modèles de scoring de crédit</li>
          </ul>
        </td>
      </tr>
      <tr>
        <td>Log Loss</td>
        <td>
          <ul>
            <li>Utilise la probabilité prédite pour chaque client, ce qui aide à calibrer le risque</li>
            <li>Pénalise sévèrement les prédictions erronées avec haute confiance</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Peut être moins intuitif pour les équipes opérationnelles</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Utile lors de la calibration du modèle pour obtenir des probabilités bien calibrées et reflétant le risque réel</li>
          </ul>
        </td>
      </tr>
    </tbody>
  </table>
</div>


Nous allons privilégier le `F1-score` comme critère.

<div class="alert alert-block alert-info">
<b>🔔 Rappels sur les pénalisations</b></p>
  <table>
    <thead>
      <tr>
        <th>Type de Régression</th>
        <th>Principe</th>
        <th>Avantages</th>
        <th>Inconvénients</th>
        <th>Quand l'utiliser ?</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Ridge (L2)</td>
        <td>Pénalise la somme des <b>carrés</b> des coefficients (L2)</td>
        <td>
          <ul>
            <li>Réduit la variance du modèle</li>
            <li>Gère bien la multicolinéarité</li>
            <li>Convient aux modèles avec beaucoup de variables peu pertinentes</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Ne réalise pas de sélection de variables</li>
            <li>Les coefficients sont réduits mais jamais annulés</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Quand toutes les variables peuvent être informatives</li>
            <li>Quand la stabilité est prioritaire à l’interprétation</li>
          </ul>
        </td>
      </tr>
      <tr>
        <td>Lasso (L1)</td>
        <td>Pénalise la somme des <b>valeurs absolues</b> des coefficients (L1)</td>
        <td>
          <ul>
            <li>Effectue une sélection automatique des variables</li>
            <li>Donne des modèles plus interprétables</li>
            <li>Peut annuler certains coefficients → simplification du modèle</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Moins stable quand les variables sont fortement corrélées</li>
            <li>Peut sélectionner arbitrairement parmi des variables similaires</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Quand on cherche un modèle simple avec peu de variables</li>
            <li>Utile en phase exploratoire pour repérer les variables importantes</li>
          </ul>
        </td>
      </tr>
      <tr>
        <td>Elastic Net (L1 + L2)</td>
        <td>Combine les pénalités L1 et L2</td>
        <td>
          <ul>
            <li>Compromis entre sélection de variables (L1) et stabilité (L2)</li>
            <li>Gère bien les variables corrélées</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Ajoute un hyperparamètre supplémentaire (`l1_ratio`)</li>
            <li>Plus complexe à ajuster</li>
          </ul>
        </td>
        <td>
          <ul>
            <li>Quand il y a beaucoup de variables, certaines corrélées, d'autres inutiles</li>
            <li>Bon choix par défaut si on hésite entre Lasso et Ridge</li>
          </ul>
        </td>
      </tr>
    </tbody>
  </table>
  <p><b>En résumé :</b></p>
    <ul>
      <li><b>Ridge (L2)</b> : Tous les coefficients restent, mais sont "rétrécis". Aucun n'est supprimé. Modèle plus stable.</li>
      <li><b>Lasso (L1)</b> : Fait une sélection de variables en mettant certains coefficients à zéro. Modèle plus simple.</li>
      <li><b>Elastic Net</b> : Équilibre entre L1 et L2, utile quand les variables sont nombreuses et corrélées.</li>
    </ul>
</div>


In [6]:
logistic_models.bayes_optimize_models(n_iter= 20, cv_folds= 5, scoring= "f1")

🔍 Optimisation en cours : Logistic Regression (L1)
🔍 Optimisation en cours : Logistic Regression (L2)
🔍 Optimisation en cours : Logistic Regression (ElasticNet)


,Modèle,Score F1 (CV),Durée (s)
1,Logistic Regression (L2),0.571900,9.15
0,Logistic Regression (L1),0.563216,20.98
2,Logistic Regression (ElasticNet),0.563181,9.51


In [7]:
# Itérations de l'optimisation bayésienne
logistic_models.plot_all_bayes_convergences()

In [8]:
# Courbes ROC
logistic_models.plot_roc_curve_interactive()

In [9]:
# Courbes Recall/Précision
logistic_models.plot_precision_recall_curve_interactive()

In [10]:
# Evolution du F1-score
dict_models = logistic_models.best_f1_by_model()

<div class="alert alert-block alert-info">
<b>🔔 Rappels sur le F1-score : interprétation selon sa valeur</b></p>

  <table border="1" style="border-collapse:collapse; width:100%; text-align:left;">
    <thead style="background-color:#f2f2f2;">
      <tr>
        <th style="padding:8px;">🔢Valeur du F1-score</th>
        <th style="padding:8px;">Interprétation Modèle</th>
        <th style="padding:8px;">Lecture Métier</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="padding:8px;"><b>≈ 1.00</b></td>
        <td style="padding:8px;">Modèle quasi-parfait : haute précision et haut rappel.</td>
        <td style="padding:8px;">Très bon modèle : détecte presque tous les défauts sans refuser inutilement.</td>
      </tr>
      <tr>
        <td style="padding:8px;"><b>0.80 – 0.99</b></td>
        <td style="padding:8px;">Excellent compromis entre détection et justesse des refus.</td>
        <td style="padding:8px;">Robuste en production si les données sont stables et bien préparées.</td>
      </tr>
      <tr>
        <td style="padding:8px;"><b>0.65 – 0.79</b></td>
        <td style="padding:8px;">Bon niveau, utilisable avec confiance.</td>
        <td style="padding:8px;">Modèle solide, peut être déployé avec suivi régulier.</td>
      </tr>
      <tr>
        <td style="padding:8px;"><b>0.50 – 0.64</b></td>
        <td style="padding:8px;">Compromis modéré, encore perfectible.</td>
        <td style="padding:8px;">Le modèle laisse passer trop de défauts ou refuse trop de bons clients.</td>
      </tr>
      <tr>
        <td style="padding:8px;"><b>0.30 – 0.49</b></td>
        <td style="padding:8px;">Modèle déséquilibré, mauvaise performance globale.</td>
        <td style="padding:8px;">Ni fiable pour la détection du risque, ni pour éviter la gêne client.</td>
      </tr>
      <tr>
        <td style="padding:8px;"><b>&lt; 0.30</b></td>
        <td style="padding:8px;">Modèle inefficace, proche d’un modèle aléatoire.</td>
        <td style="padding:8px;">Modèle non utilisable pour l’octroi de crédit.</td>
      </tr>
    </tbody>
  </table>
</div>


#### 6.3 Focus sur le meilleur modèle (au sens du F1-score)

#### 6.4 Evaluer le modèle (sélectionner le modèle qui maximise le F1-score)

  <div class="alert alert-block alert-info">
  <h4><b>Recall et Précision dans un modèle de score d’octroi de crédit</b></h4>

  <h5><u>Recall</u></h5>
  <p>
    <b>Formule :</b><br>
    Recall = VP / (VP + FN) = Part des défauts correctement détectés par le modèle parmi tous les défauts réels.
  </p>
  <p>
    <b>Interprétation :</b><br>
    Parmi les clients qui auraient réellement fait défaut, combien ont été correctement prédits comme risqués ?
  </p>
  <p>
    <b>Exemple :</b><br>
    Un recall de 10% signifie que le modèle détecte seulement 10% des clients risqués. Il laisse passer 90% des défauts potentiels.
  </p>
  <p>
    <b>Enjeu métier :</b><br>
    Maximiser le recall, c’est minimiser le nombre de crédits accordés à des clients qui ne rembourseront pas.<br>
    <i>👉 La banque cherche ici à limiter au maximum les pertes financières liées aux impayés, quitte à refuser certains bons clients.</i>
  </p>

  <h5><u>Précision</u></h5>
  <p>
    <b>Formule :</b><br>
    Précision = VP / (VP + FP) = Part des clients correctement identifiés comme risqués parmi ceux prédits comme tels.
  </p>
  <p>
    <b>Interprétation :</b><br>
    Parmi tous les clients prédits comme risqués, combien sont effectivement de vrais clients à risque ?
  </p>
  <p>
    <b>Exemple :</b><br>
    Une précision de 70% signifie que 30% des refus concernent des clients qui auraient bien remboursé (faux positifs).
  </p>
  <p>
    <b>Enjeu métier :</b><br>
    Maximiser la précision, c’est éviter de refuser un crédit à un bon client.<br>
    <i>👉 La banque veut ici préserver sa relation client et son image de marque.</i>
  </p>

  <h5><u>Trade-off entre Recall et Précision</u></h5>
  <table border="1" style="border-collapse:collapse; width:100%; text-align:left;">
    <thead>
      <tr style="background-color:#f2f2f2;">
        <th style="padding:8px;">🎯 Objectif métier</th>
        <th style="padding:8px;">📈 Priorité métrique</th>
        <th style="padding:8px;">💬 Conséquences</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="padding:8px;">Limiter au maximum les défauts de paiement</td>
        <td style="padding:8px;">Recall élevé</td>
        <td style="padding:8px;">Risque de refuser des clients solvables (gêne client)</td>
      </tr>
      <tr>
        <td style="padding:8px;">Éviter les refus injustifiés / préserver l'image</td>
        <td style="padding:8px;">Précision élevée</td>
        <td style="padding:8px;">Risque d’accorder un crédit à un client risqué</td>
      </tr>
      <tr>
        <td style="padding:8px;">Trouver un équilibre entre risque et opportunité</td>
        <td style="padding:8px;">F1-score (compromis)</td>
        <td style="padding:8px;">Équilibre entre détection du risque et satisfaction client</td>
      </tr>
    </tbody>
  </table>

  <p style="margin-top:15px;">
    ✅ <b>Conclusion :</b> Le choix de la métrique dépend des priorités stratégiques de la banque : <br>
    - <i>Plutôt risk-averse ?</i> → Maximiser le recall.<br>
    - <i>Plutôt orientée expérience client ?</i> → Maximiser la précision.<br>
    - <i>Besoin d’un bon compromis ?</i> → Suivre le F1-score.
  </p>


In [11]:
dict_best_model = logistic_models.get_best_model() # modèle qui maximise le critère
best_model_name = dict_best_model["model_name"]

Meilleur modèle : Logistic Regression (L2)
Score (F1 CV) : 0.5719


In [12]:
logistic_models.evaluate_model(best_model_name)

<div class="alert alert-block alert-success">
  <h4>📊 Interprétation des performances du modèle de scoring</h4>

  <p><b>📌 Répartition de la variable cible :</b> 70% de <i>non défaut</i> (clients fiables) et 30% de <i>défaut</i> (clients à risque).</p>
  <p>Ce déséquilibre implique qu’un bon modèle ne peut pas se contenter d’une forte accuracy : il doit surtout bien détecter les défauts tout en limitant les faux refus.</p>

  <h5>⚙️ Métriques globales</h5>
  <table border="1" style="border-collapse: collapse; width: 100%; text-align: left;">
    <thead style="background-color:#f2f2f2;">
      <tr>
        <th style="padding:8px;">Métrique</th>
        <th style="padding:8px;">Valeur</th>
        <th style="padding:8px;">Interprétation</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="padding:8px;">Accuracy</td>
        <td style="padding:8px;">0.77</td>
        <td style="padding:8px;">
          77% des prédictions sont correctes. À interpréter avec prudence car un modèle naïf qui prédirait toujours "non défaut" atteindrait déjà 70%.
        </td>
      </tr>
      <tr>
        <td style="padding:8px;">ROC AUC</td>
        <td style="padding:8px;">0.7452</td>
        <td style="padding:8px;">
          Bonne capacité à discriminer les bons des mauvais payeurs. Score satisfaisant dans un contexte de déséquilibre.
        </td>
      </tr>
    </tbody>
  </table>

  <h5 style="margin-top:20px;">🏷️ Métriques par classe</h5>
  <table border="1" style="border-collapse: collapse; width: 100%; text-align: left;">
    <thead style="background-color:#f2f2f2;">
      <tr>
        <th style="padding:8px;">Classe</th>
        <th style="padding:8px;">🎯 Précision</th>
        <th style="padding:8px;">🔍 Recall</th>
        <th style="padding:8px;">⚖️ F1-score</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="padding:8px;"><b>Non défaut (classe 0)</b></td>
        <td style="padding:8px;">
          <b>0.856</b><br>
          → 86% des clients prédits comme fiables le sont réellement. Le modèle évite efficacement les faux refus.
        </td>
        <td style="padding:8px;">
          <b>0.807</b><br>
          → Le modèle capte 81% des clients qui remboursent. Il maintient la satisfaction des clients solvables.
        </td>
        <td style="padding:8px;">
          <b>0.831</b><br>
          → Très bon équilibre sur cette classe majoritaire. Peu d’erreurs, bon confort métier.
        </td>
      </tr>
      <tr>
        <td style="padding:8px;"><b>Défaut (classe 1)</b></td>
        <td style="padding:8px;">
          <b>0.603</b><br>
          → Sur 100 clients considérés comme risqués, 60 le sont réellement. Taux de refus injustifiés à surveiller.
        </td>
        <td style="padding:8px;">
          <b>0.683</b><br>
          → 68% des vrais défauts sont détectés. Bon score dans un contexte déséquilibré, mais perfectible.
        </td>
        <td style="padding:8px;">
          <b>0.641</b><br>
          → Compromis raisonnable entre détection et justesse. Des défauts échappent encore au modèle.
        </td>
      </tr>
    </tbody>
  </table>

  <h4 style="margin-top: 20px;">🚀 Axes pour améliorer la performance</h4>
  <ul>
    <li><b>Ajuster le seuil de décision</b> pour privilégier le rappel (ex : seuil &lt; 0.5).</li>
    <li><b>Enrichir les variables explicatives</b> (demander d'autres informations au métier, faire des croisements entre variables, appliquer des transformations aux variables).</li>
    <li><b>Tester d’autres modèles</b> plus performants : Random Forest, XGBoost, LightGBM.</li>
    <li><b>Travailler avec les métiers</b> pour ajuster le compromis recall / précision selon la stratégie risque / image client.</li>
  </ul>
</div>


#### 6.4.2 Déterminer un seuil manuellement (avec une approche métier)

In [13]:
logistic_models.plot_metrics_by_threshold(best_model_name)
df_metrics = logistic_models.get_metrics_by_threshold(best_model_name)

<div class="alert alert-block alert-warning">
<b>Question 30:</b> Nous souhaitons refuser moins de 30% de crédits à tort,  quel seuil choisir ?
</div>

<div class="alert alert-block alert-success">
<b> Réponse :</b> Seuil où Précision>=70%
</div>

In [14]:
# On cherche à maximiser le recall pour une précision>=0.7
df_metrics[df_metrics["precision"]>=0.7].sort_values("recall",ascending=False).head(1)

,threshold,precision,recall,f1
64,0.643535,0.724138,0.35,0.47191


<div class="alert alert-block alert-warning">
<b>Question 31:</b> Nous souhaitons détecter au moins 75% des défauts de paiement , quel seuil choisir ?
</div>

<div class="alert alert-block alert-success">
<b> Réponse :</b> Seuil où Recall >=75%
</div>

In [15]:
df_metrics[df_metrics["recall"]>=0.75].sort_values("precision",ascending=False).head(1)

,threshold,precision,recall,f1
21,0.217879,0.46875,0.75,0.576923


### 7. Modèles à base d'arbres

In [ ]:
list_models = ["Random Forest", "Gradient Boosting", "LightGBM"] # liste des modélisations à tester
trees_models = ModelComparator(y_train= y_train, X_train= X_train_with_norm, y_test= y_test, X_test= X_test_with_norm, list_models= list_models, random_state= random_state) # initier la class permettant la modélisation
trees_models.bayes_optimize_models(n_iter= 20, cv_folds= 5, scoring= "f1")

🔍 Optimisation en cours : Random Forest


c:\Users\ADUBOIS\OneDrive - Micropole\cours_scoring\.venv_cours_scoring\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning:

The objective has been evaluated at point [np.True_, np.int64(15), np.str_('sqrt'), np.int64(1), np.int64(100)] before, using random point [True, np.int64(11), 'sqrt', np.int64(9), np.int64(223)]

c:\Users\ADUBOIS\OneDrive - Micropole\cours_scoring\.venv_cours_scoring\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning:

The objective has been evaluated at point [np.True_, np.int64(15), np.str_('sqrt'), np.int64(1), np.int64(100)] before, using random point [True, np.int64(8), 'log2', np.int64(1), np.int64(300)]

c:\Users\ADUBOIS\OneDrive - Micropole\cours_scoring\.venv_cours_scoring\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning:

The objective has been evaluated at point [np.True_, np.int64(15), np.str_('sqrt'), np.int64(1), np.int64(100)] before, using random point [True, np.int64(4), 'sqrt', np.int64(6), np.int64(264)

🔍 Optimisation en cours : Gradient Boosting
🔍 Optimisation en cours : LightGBM
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

,Modèle,Score F1 (CV),Durée (s)
2,LightGBM,0.566904,32.33
1,Gradient Boosting,0.553101,22.16
0,Random Forest,0.504801,14.39


In [17]:
# Itérations de l'optimisation bayésienne
trees_models.plot_all_bayes_convergences()

In [18]:
# Courbes ROC
trees_models.plot_roc_curve_interactive()

In [19]:
# Courbes Recall/Précision
trees_models.plot_precision_recall_curve_interactive()

In [20]:
# Evolution du F1-score
dict_models = trees_models.best_f1_by_model()

In [21]:
dict_best_model_trees = trees_models.get_best_model() # modèle qui maximise le critère
best_model_name_trees = dict_best_model_trees["model_name"]

Meilleur modèle : LightGBM
Score (F1 CV) : 0.5669


In [22]:
trees_models.evaluate_model(best_model_name_trees)

**Aide à la sélection "métier" du seuil de score**

In [23]:
trees_models.plot_metrics_by_threshold(best_model_name_trees)
df_metrics_trees = trees_models.get_metrics_by_threshold(best_model_name_trees)